# Investigation of Grover's  algorithm with noise

Based on 
https://pennylane.ai/qml/demos/tutorial_qjit_compile_grovers_algorithm_with_catalyst

But modified to use the density matrix (https://docs.pennylane.ai/en/stable/code/api/pennylane.density_matrix.html).

Some good sources for the theory behind Grover's algorithm

*  https://quantum.cloud.ibm.com/learning/en/courses/fundamentals-of-quantum-algorithms/grover-algorithm/grover-algorithm-description

  Information about noise models

*  https://pennylane.ai/qml/demos/tutorial_how_to_use_noise_models

The starting point is searching for x such that $ f(\mid x > ) = - \mid x > $


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml



In [4]:
def equal_superposition(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)


In [5]:
def oracle(wires, omega):
    """
        Apply the Oracle
    """
    qml.FlipSign(omega, wires=wires)



In [6]:
def num_grover_iterations(N, M):
    """
     Set the number of iterations for the Grover's algorithm,
    """
    return np.ceil(np.sqrt(N / M) * np.pi / 4).astype(int)



In [7]:
def grover_circuit(num_qubits):
    """
      Circuit for the Grover's algorithm
    """
    wires = list(range(num_qubits))
    omega = np.array([np.zeros(num_qubits), np.ones(num_qubits)])


    M = len(omega)
    N = 2**num_qubits

    # Initial state preparation
    equal_superposition(wires)

    # Grover iterations
    for _ in range(num_grover_iterations(N, M)):
        for omg in omega:
            oracle(wires, omg)
        qml.templates.GroverOperator(wires)

    #  return a state
    return qml.state()
    


## Apply the circuit for Grover's algorithm

In [8]:
#NUM_QUBITS = 12
NUM_QUBITS = 6

# use the density matrix
dev = qml.device("default.mixed", wires=NUM_QUBITS)

@qml.qnode(dev)
def circuit_default_qubit():
    return grover_circuit(NUM_QUBITS)


results = circuit_default_qubit()


##  Introduce some noise sources

In [9]:
fcond3 = qml.noise.op_eq("Hadamard") & qml.noise.wires_in([0, 1])

def noise3(op, **kwargs):
    qml.RX(np.pi / 16, op.wires)
    qml.apply(op)
    qml.RY(np.pi / 8, op.wires)



In [10]:

metadata = dict()  #
noise_model = qml.NoiseModel(
    {fcond3: noise3}, **metadata
)
print(noise_model)



NoiseModel({
    OpEq(Hadamard) & WiresIn([0, 1]): noise3
})


##  Run the circuit with noise included

In [11]:
noisy_circuit = qml.add_noise( grover_circuit, noise_model)

@qml.qnode(dev)
def noise_circuit_default_qubit():
    return noisy_circuit(NUM_QUBITS)


results_noise = noise_circuit_default_qubit()


### Compare the noise and noiseless circuits

Compare the density matrix from the noiseless and noisy channel using the fidelity

* https://en.wikipedia.org/wiki/Fidelity_of_quantum_states
* https://docs.pennylane.ai/en/stable/code/api/pennylane.math.fidelity.html


In [14]:

noisy_res = np.round(qml.math.fidelity(results, results_noise ), 8)
res_res = np.round(qml.math.fidelity(results, results ), 8)

print(f"Ideal v/s Noisy:  {noisy_res}")
print(f"Ideal v/s Ideal:  {res_res}")

Ideal v/s Noisy:  0.44186913
Ideal v/s Ideal:  1.0000001
